In [4]:
import sqlite3

conn = sqlite3.connect("./database/database.db")
cur = conn.cursor()
sql = "DELETE FROM T車両台帳;"
cur.execute(sql)
conn.commit()
sql = """
    SELECT
    company_use_number,
    classification,
    maker,
    model_number,
    body_number,
    modelYear,
    car_size,
    load_capacity,
    specification,
    riding_capacity,
    fuel_type,
    update_date
    FROM TBase_Vehicle
    ORDER BY id;
    """
cur.execute(sql)
result = cur.fetchall()
i = 0
for c in result:
    i += 1
    sql = f"INSERT INTO T車両台帳 VALUES({i}, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"
    cur.execute(sql, c)
    conn.commit()

In [ ]:
import sqlite3

conn = sqlite3.connect("database/database.db")
cur = conn.cursor()
sql = """
DELETE FROM T車両台帳 WHERE company_use_number = '123';
DELETE FROM T登録番号 WHERE company_use_number = '123';
DELETE FROM T車両履歴 WHERE company_use_number = '123';
"""
cur.executescript(sql)
conn.commit()

In [3]:
import sqlite3

# テーブルバックアップ
conn = sqlite3.connect("database/database.db")
cur = conn.cursor()
sql = """
    DELETE FROM TBK車両台帳;
    DELETE FROM TBK車両履歴;
    DELETE FROM TBK登録番号;
    INSERT INTO TBK車両台帳 SELECT * FROM T車両台帳;
    INSERT INTO TBK車両履歴 SELECT * FROM T車両履歴;
    INSERT INTO TBK登録番号 SELECT * FROM T登録番号;
    """
cur.executescript(sql)
conn.commit()

In [1]:
import sqlite3

# バックアップデータ戻し
conn = sqlite3.connect("database/database.db")
cur = conn.cursor()
sql = """
    DELETE FROM T車両台帳;
    DELETE FROM T車両履歴;
    DELETE FROM T登録番号;
    INSERT INTO T車両台帳 SELECT * FROM TBK車両台帳;
    INSERT INTO T車両履歴 SELECT * FROM TBK車両履歴;
    INSERT INTO T登録番号 SELECT * FROM TBK登録番号;
    """
cur.executescript(sql)
conn.commit()

In [ ]:
import sqlite3

# TW系テーブルクリア
conn = sqlite3.connect(r"database\database.db")
cur = conn.cursor()
sql = """
    DELETE FROM TPrintWork;
    DELETE FROM TW内訳別申請台数;
    DELETE FROM TW内訳別保有台数;
    DELETE FROM TW車格別保有台数;
    DELETE FROM TW申請車両;
    """
cur.executescript(sql)
conn.commit()

In [9]:
import datetime
from dateutil.relativedelta import relativedelta

str_date = '2023/05/01'
target_date = datetime.datetime.strptime(str_date, '%Y/%m/%d')
result_date = target_date + relativedelta(months=-3)
three_month_ago = str(format(result_date, '%Y/%m/%d'))
print(three_month_ago)

2023/02/01


In [16]:
import sqlite3

conn = sqlite3.connect(r"database\database.db")
cur = conn.cursor()
sql = """
    SELECT circumstances FROM T車両履歴
    WHERE department = '03'
    AND implementation_date <= '2023/10/01'
    GROUP by circumstances;
    """
cur.execute(sql)
result = cur.fetchall()
print(result)
circ_list = []
for circ in result:
    circ_list.append(circ[0])
if "B" in circ_list:
    j = True
else:
    j = False
print(j) 

[('A',), ('F',), ('T',)]
False


In [2]:
import sqlite3

DATABASE = r"database\database.db"

sql = f"""
    SELECT a.implementation_date,
    CASE
    WHEN a.circumstances = 'A' THEN '導入'
    WHEN a.circumstances = 'F' THEN '移動（元）'
    WHEN a.circumstances = 'T' THEN '移動（先）'
    WHEN a.circumstances = 'D' THEN '廃止'
    END as situation,
    a.department,
    b.name
    FROM T車両履歴 as a
    LEFT JOIN M部署 as b
    on a.department = b.code
    WHERE a.company_use_number = '216161'
    ORDER by id;
    """
conn = sqlite3.connect(DATABASE)
cur = conn.cursor()
cur.execute(sql)
result = cur.fetchall()
print(result)

[('2022/01/01', '導入', '03', '戸田'), ('2023/10/02', '移動（元）', '03', '戸田'), ('2023/10/02', '移動（先）', '07', '茨城')]


In [46]:
import sqlite3
import openpyxl
import os

DATABASE = "database/database.db"
YEAR = 2023
FOLDER = r"document\月末指針"

data_org = os.path.join(FOLDER, f"G_{str(YEAR)}.xlsx")
wb = openpyxl.load_workbook(data_org)
sheetname_dict = {"茨城": "07", "足立": "09", "上三川": "22", "栃木": "02",
                  "名古屋": "13", "共配": "09", "千葉": "04", "神奈川": "08", "群馬": "05"}


# T燃料使用量の最新のidを取得
def get_new_id():
    sql_0 = "SELECT * FROM T燃料使用量;"
    sql_1 = "SELECT max(id) FROM T燃料使用量;"
    conn = sqlite3.connect(DATABASE)
    cur = conn.cursor()
    cur.execute(sql_0)
    if len(cur.fetchall()) == 0:
        new_number = 1
    else:
        cur.execute(sql_1)
        result = cur.fetchone()
        new_number = result[0] + 1
    return new_number

    
# 戸田以外の「月末指針」Excelデータを取得
i = get_new_id()
rows = []
for k, v in sheetname_dict.items():
    ws = wb[k]
    c = 1
    for m in [4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3]:
        r = 3
        while True:
            if ws.cell(r, c).value is None:
                break
            else:
                y = YEAR + 1 if m < 4 else YEAR
                section = (i, y, m,
                        ws.cell(r, c).value,
                        v,
                        0.0 if ws.cell(r, c+2).value is None else float(ws.cell(r, c+2).value),
                        0.0 if ws.cell(r, c+4).value is None else float(ws.cell(r, c+4).value)
                        )
                rows.append(section)
                i += 1
                r += 1
        c += 5

# 戸田の「月末指針」Excelデータを取得
ws = wb["戸田"]
c = 1
for m in [4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3]:
    r = 3
    while True:
        if ws.cell(r, c).value is None:
            break
        else:
            y = YEAR + 1 if m < 4 else YEAR
            section = (i, y, m,
                    ws.cell(r, c).value,
                    "03",
                    0.0 if ws.cell(r, c+2).value is None else float(ws.cell(r, c+2).value),
                    0.0 if ws.cell(r, c+6).value is None else float(ws.cell(r, c+6).value)
                    )
            rows.append(section)
            i += 1
            r += 1
    c += 7

# T燃料使用量テーブルにデータを書き込み
sql = """
    INSERT INTO T燃料使用量 VALUES(?, ?, ?, ?, ?, ?, ?)
    """
conn = sqlite3.connect(DATABASE)
cur = conn.cursor()
cur.executemany(sql, rows)
conn.commit()

In [6]:
import openpyxl

wb = openpyxl.load_workbook("月末指針_2022.xlsx")
ws = wb["茨城"]

print(type(ws.cell(29,1).value))

<class 'NoneType'>


In [16]:
import sqlite3

DATABASE = "database/database.db"

MAXYEARMONTH = ""


def get_db_connection():
    connection = sqlite3.connect(DATABASE)
    connection.row_factory = sqlite3.Row
    return connection

def get_mileage(company_use_number, year, month):
        sql_min = f"""SELECT company_use_number,
                min(year || substr('00' || month, -2, 2)) as yearmonth,
                min(meter) as min_meter
                FROM T燃料使用量
                WHERE company_use_number = '{company_use_number}'
                GROUP by company_use_number;"""
        sql_max = f"""SELECT department, company_use_number,
                max(year || substr('00' || month, -2, 2)) as yearmonth,
                max(meter) as max_meter
                FROM T燃料使用量
                WHERE company_use_number = '{company_use_number}'
                AND year = (SELECT max(year) FROM T燃料使用量 
                WHERE company_use_number = '{company_use_number}'
                AND year <= {year} AND meter > 0)
                AND month = (SELECT max(month) FROM T燃料使用量 
                WHERE company_use_number = '{company_use_number}'
                AND month <= {month} AND meter > 0)
                GROUP by company_use_number;"""
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute(sql_min)
        result = cur.fetchone()
        meter_min = result["min_meter"]
        cur.execute(sql_max)
        result = cur.fetchone()
        meter_max = result["max_meter"]
        mileage = meter_max - meter_min
        print(f"自: {meter_min}　至: {meter_max}　走行距離: {mileage}")

get_mileage("610109", 2023, 8)

自: 0　至: 455704　走行距離: 455704


In [34]:
import sqlite3

DATABASE = "database/database.db"


def get_db_connection():
    connection = sqlite3.connect(DATABASE)
    connection.row_factory = sqlite3.Row
    return connection


class FuelEfficiency:
    def __init__(self, year, company_use_number):
        self.company_use_number = company_use_number
        self.this_year = year

    def lastest_meter(self):
        sql = f"""
            select * from
            (select id, year || substr('0' || month, -2, 2) as year_m, 
            company_use_number, meter, fuel_usage
            from T燃料使用量
            where company_use_number ='{self.company_use_number}'
            and year_m >= '{self.this_year}04'
            and year_m <= '{str(self.this_year + 1)}03'
            and meter != 0)
            where id = 
            (select max(id) from
            (select * from T燃料使用量
            where company_use_number ='{self.company_use_number}'
            and year_m >= '{self.this_year}04'
            and year_m <= '{self.this_year + 1}03'
            and meter != 0));
            """
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute(sql)
        result = cur.fetchone()
        return result

    def start_meter(self):
        # 本年度以前の履歴があるか
        sql = f"""
            select * from T燃料使用量
            where company_use_number = '{self.company_use_number}'
            and meter != 0 and year || substr('0' || month, -2, 2) <= '{self.this_year}03';
            """
        conn = sqlite3.connect(DATABASE)
        cur = conn.cursor()
        cur.execute(sql)
        if len(cur.fetchall()) != 0:
            sql = f"""
                select id, year || substr('0' || month, -2, 2) as year_m,
                company_use_number, meter, fuel_usage
                from T燃料使用量
                where id = (select max(id) from
                (select id from T燃料使用量 where company_use_number ='{self.company_use_number}'
                and meter != 0 and year || substr('0' || month, -2, 2) <= '{self.this_year}03'));
                """
        else:
            sql = f"""
                select id, year || substr('0' || month, -2, 2) as year_m, 
                company_use_number, meter, fuel_usage
                from T燃料使用量
                where id = (select min(id) from
                (select id from T燃料使用量 where company_use_number ='{self.company_use_number}'
                and meter != 0 and year || substr('0' || month, -2, 2) >= '{self.this_year}04'));
                """
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute(sql)
        result = cur.fetchone()
        return result

    def fuel_efficiency(self):
        sql = f"""
            select company_use_number, sum(fuel_usage) as fuel from T燃料使用量
            where company_use_number ='{self.company_use_number}'
            and year || substr('0' || month, -2, 2) >= '{self.this_year}04'
            and year || substr('0' || month, -2, 2) <= '{self.this_year + 1}03'
            group by company_use_number
            """
        
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute(sql)
        result = cur.fetchone()
        return result


if __name__ == "__main__":
    fe = FuelEfficiency(2023, "623141")
    # 直近の走行距離を取得
    data = fe.lastest_meter()
    lastest_meter = data["meter"]
    # 前年度末の走行距離を取得
    data = fe.start_meter()
    start_meter = data["meter"]
    # 本年度の年間走行距離を取得
    annual_mileage = lastest_meter - start_meter
    print(f"年間走行距離: {annual_mileage}km")
    # 本年度の年間燃料使用量を取得
    fuel_efficiency = fe.fuel_efficiency()
    print(f"燃料使用量: {fuel_efficiency['fuel']}ℓ")

年間走行距離: 7804km
燃料使用量: 2050.0ℓ


In [51]:
import sqlite3

DATABASE = "database/database.db"


def get_last_insert():
    sql = """
        select b.name, max(a.year || substr('0' || a.month, -2, 2)) as odr from T燃料使用量 as a
        left join M部署 as b on a.department = b.code
        group by b.name
        """
    conn = sqlite3.connect(DATABASE)
    cur = conn.cursor()
    cur.execute(sql)
    result = cur.fetchall()
    for k, v in result:
        print(f"{k} {v[:4]}年{v[4:]}月")

get_last_insert()

千葉 2023年08月
名古屋 2023年08月
宇都宮 2023年08月
戸田 2023年08月
栃木 2023年08月
神奈川 2023年08月
群馬 2023年03月
茨城 2023年07月
足立 2023年09月
